# XGBoost notebook

## Overview

In this tutorial notebook, we will run through the process of fitting [XGBoost](https://xgboost.readthedocs.io/en/stable/) models to tabular data, in order to predict surface turbulent fluxes over sea ice in the Arctic. This application to polar turbulent fluxes, inspired by the work of [Cummins et al. (2023)](https://doi.org/10.1029/2023GL105698) and [Cummins et al. (2024)](https://doi.org/10.1007/s10546-023-00852-8), is an example of a parametrization problem that is hard to solve with traditional physics, and showcases the ease with which modern boosting methods allow us to obtain performant models.

## Prerequisites

Readers are assumed to have basic familiarity with the programming language [Python](https://www.python.org/), and to have already completed the [tutorial notebook on Random Forests](https://github.com/cemac/LIFD_RandomForests) from the [Leeds Institute for Fluid Dynamics (LIFD)](https://fluids.leeds.ac.uk/). Required data are supplied with the present notebook in the GitHub repository. You will need to have the following Python packages installed:

* pandas
* NumPy
* XGBoost
* Matplotlib
* scikit-learn

## Brief introduction to gradient boosting and XGBoost

XGBoost (eXtreme Gradient Boosting) is a machine-learning (ML) library for training gradient-boosting models. [Gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) (or simply "boosting") is a strategy for building ensemble estimators from [decision trees](https://en.wikipedia.org/wiki/Decision_tree_learning). While similar to [Random Forest](https://en.wikipedia.org/wiki/Random_forest) in this regard, boosting methods are fundamentally different in the ways the trees are grown and their predictions combined.

### Random Forest

In a Random Forest, the training of individual trees happens in parallel. Each tree is trained on a randomly resampled version of the original dataset, in a process known as [bootstrap aggregating](https://en.wikipedia.org/wiki/Bootstrap_aggregating), or "bagging". Bagging ensures that the trees in the Random Forest are sufficiently different from one another. It is this heterogeneity of the trees which is key to the Random Forest's predictive power. Bagging is particularly effective at reducing the [variance](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff) (sensitivity to perturbations of the training data) of an unstable algorithm, such as a decision tree, without increasing its [bias](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff). Random Forests typically use deep decision trees as their constituent ensemble members, in order to reduce the bias as much as possible. The bagging / Random Forest idea can be crudely summarized as follows:

* A complicated and unstable model (e.g. a decision tree) can achieve low bias but at the cost of high variance (i.e. will overfit).
* By averaging predictions from a large number of overfitted models, each trained on a different realization of the dataset, we can effectively control the variance.

The end result should be a Random Forest model with low bias (due to the deep trees) and a greatly reduced propensity to overfit (due to the bagging).

### Gradient Boosting

Unlike the Random Forest, gradient boosting builds an ensemble of trees sequentially. Trees are added to the ensemble, one at a time, until some stopping criterion is satisfied, often simply a predetermined number of trees. The training is done sequentially, because the dataset used to train each tree depends on the trees that came before it. This is fundamentally different from the Random Forest, where the trees are independent of each other by design (as far as possible). In gradient boosting, each new tree in the ensemble is trained on the prediction errors (residuals) of its predecessors. The idea is that by continually adding trees, the model is able to correct its previous mistakes. Gradient boosting can often outperform Random Forest in classification and regression tasks.

Gradient boosting typically uses shallow trees, unlike the deep trees used by Random Forest. A shallow tree has low variance but high bias. By training a sequence of shallow trees, each trained to correct the previous trees' mistakes, the bias of the ensemble can be reduced almost arbitrarily, but at the cost of increasing variance. The number of trees is therefore an important hyperparameter. To prevent overfitting, boosted trees require [regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)). In traditional gradient boosting, the regularization is achieved primarily through the use of a [learning rate](https://en.wikipedia.org/wiki/Learning_rate) parameter. Instead of each new tree's prediction being added directly to the prediction of the ensemble, it is first scaled by a number between zero and one (the learning rate). This type of regularization is known as ["shrinkage"](https://en.wikipedia.org/wiki/Shrinkage_(statistics)) and is also used in other learning algorithms such as [linear regression](https://en.wikipedia.org/wiki/Linear_regression) and [artificial neural networks](https://en.wikipedia.org/wiki/Neural_network_(machine_learning)).

### XGBoost

XGBoost is a particular variant of gradient boosting, which has become popular in recent years due to its state-of-the-art predictive accuracy and speed. The statistical model used in XGBoost benefits from multiple additional types of regularization, controllable through model hyperparameters, as well as robust rules for handling missing data. XGBoost is also a highly optimized and flexible software library, with support for parallel processing via GPUs and distributed computing systems. In short, XGBoost is not just a modelling methodology, but a complete computational system for supervised machine learning.

Note that competitors to XGBoost have appeared in recent years, e.g. [CatBoost](https://catboost.ai/) and Microsoft's [LightGBM](https://lightgbm.readthedocs.io/en/latest/index.html), so it might be worth trying multiple boosting libraries on your particular problem to see which performs best.

## Data preparation

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV

In this example we will be using meteorological data from the [Surface Heat Budget of the Arctic Ocean (SHEBA) campaign](https://www.eol.ucar.edu/field_projects/sheba). The SHEBA field experiment, conducted from 1997 to 1998, aimed to document, understand, and predict the physical processes that determine the surface energy budget and the sea-ice mass balance in the Arctic. The experiment was motivated by significant changes in the Arctic, hypothesized to be a combination of poorly understood natural modes of variability and anthropogenic greenhouse warming.

Note that this is [Ola Persson's composited version of the dataset](https://doi.org/10.5065/D6PN93R6), described in [his 2011 paper](https://doi.org/10.1007/s00382-011-1196-9), which combines data from several observation sites in the SHEBA ice camp and elsewhere to produce a largely complete hourly record of the turbulent fluxes and meteorological variables.

In [ ]:
# Read composited SHEBA dataset
df = pd.read_csv('sheba_composite_data_qb2.txt', sep='\\s+', skiprows=range(8), header=[0,1])

# Extract variable names
variable_names = [column[0] for column in df.columns]
variable_units = [column[1] for column in df.columns]

# Index columns by variable only
df.columns = variable_names

# Index rows by datetime
base_date = pd.Timestamp('1997-01-01')
df['time'] = [base_date + pd.Timedelta(days=jd) for jd in df['jd']]
df.set_index('time', inplace=True)
df.drop(columns='jd', inplace=True)

We will attempt to predict friction velocity, $u_\star$ (m/s), and surface turbulent fluxes of sensible and latent heat, $H_s$ and $H_l$ (W/m<sup>2</sup>). These are our "target" variables. To make our predictions, we will use the following input variables or "features":

| Feature | Description | Units |
| ------- | ----------- | ----- |
| `press` | Air pressure | hPa |
| `t2`    | 2m air temperature | Deg. Celsius |
| `t10`    | 10m air temperature | Deg. Celsius |
| `q2`    | 2m specific humidity | g/kg |
| `q10`    | 10m specific humidity | g/kg |
| `ws2`    | 2m windspeed | m/s |
| `ws10`    | 10m windspeed | m/s |
| `snwdpthi`    | Snow depth | m |

Note that the heights are nominal and were subject to change throughout the SHEBA campaign due to accretion of snow. By including snow depth as a feature, the model may be able to take this into account.

In [ ]:
# Targets and features
targets = ['usmed','hsmed','hlmed']
features = ['press','t2','t10','q2','q10','ws2','ws10','snwdpthi']

XGBoost is able to deal with cases where some of the features are missing, so we don't have to remove those cases. However, we do need to remove cases where one or more of the target variables is missing.

In [ ]:
# Remove rows where targets missing
df = df.dropna(subset=targets)

The composited SHEBA dataset pools information from various sensors to produce a more complete dataset. Unfortunately, it still contains some bad data. Let's replace the obviously erroneous data with missing values. This way, XGBoost will still be able to make use of the non-missing features for those cases.

In [ ]:
# Remove negative values of ustar
df = df[df['usmed'] > 0.]

# Replace negative humidities and unrealistic windspeeds by NaN
df.loc[df['q2'] <= 0., 'q2'] = np.NaN
df.loc[df['ws10'] > 100., 'ws10'] = np.NaN
df.loc[df['ws10'] < 1e-6, 'ws10'] = np.NaN

There are also some suspicious windspeed gradients in the dataset. We'll leave these alone for now, as it's not obvious what the criterion for inclusion should be. We know that the data will contain some observational noise, so we will have to be careful that our model doesn't overfit. Model validation will be essential.

## Model training

As of version 2.0, XGBoost is capable of building multi-output tree models (to predict all our targets at once). For simplicity, in this notebook we will build models for each target separately, starting with the sensible heat flux $H_s$ (W/m<sup>2</sup>). We will start by splitting our dataset into a training set and a testing set. The `train_test_split` function from [scikit-learn](https://scikit-learn.org/stable/) makes this easy for us.

In [ ]:
# Build training and testing sets
X, y = df[features], df[targets[1]] # sensible heat flux (set to 0 or 2 to try predicting u_star or H_l)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

There are two APIs for building XGBoost models: the native API and the [scikit-learn](https://scikit-learn.org/stable/) API. The native API is older and offers a wider range of features, but is slightly more complicated to learn. In particular, to use the native API it's necessary to use XGBoost's own custom data structure called the "DMatrix". The [scikit-learn](https://scikit-learn.org/stable/) API will be familiar, as we already used it for model training in the Random Forest notebook. Here we will use the [scikit-learn](https://scikit-learn.org/stable/) API.

### Default settings

Let's first train an XGBoost model with the default hyperparameter settings and see how it performs.

In [ ]:
# Initialize an XGBoost model with default parameters
model_default = xgb.XGBRegressor()

# Fit model to training set
model_default.fit(X_train, y_train)

# Make predictions on training set
y_pred_train = model_default.predict(X_train)

# Compute Root Mean Squared Error (RMSE) and Coefficient of Determination (R^2)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
r2 = r2_score(y_train, y_pred_train)
print(f'RMSE: {rmse:.2f} W/m2')
print(f'R^2: {r2:.3f}')

Wow... an R<sup>2</sup> of 94 % is impressive! Of course, this figure is computed from the training set, so is probably an optimistic estimate of model performance. Even so, using only the default parameter settings, our XGBoost model has explained almost all the variance in the training data. It's a very flexible model.

Now let's look at the predictions plotted against the true values in the training set.

In [ ]:
# Plot residuals on training set
fig, ax = plt.subplots()
ax.plot(y_train, y_pred_train, 'bo')
ax.axline((0, 0), slope=1, label='y=x')
ax.set_title('Default parameters, training set')
ax.set_xlabel('H_s true')
ax.set_ylabel('H_s predicted')
ax.legend();

It's almost a perfect fit to the training set. Note the conservative bias in the tails: the model underpredicts the highest fluxes and overpredicts the lowest fluxes. This is due to the regularization built into the XGBoost algorithm. Regularization is enabled even with the default hyperparameter settings.

### Hyperparameter tuning

Hopefully, we can do better than the default hyperparameter settings. XGBoost has many [tuneable hyperparameters](https://xgboost.readthedocs.io/en/latest/parameter.html). Here are six important ones:

| Parameter         | Range                | Default value | Description                                                                                     |
|-------------------|----------------------|---------------|-------------------------------------------------------------------------------------------------|
| n_estimators      | Positive integer     | 100           | The number of boosting rounds (trees) in the XGBoost model. Increasing may improve performance. |
| learning_rate     | (0, 1]               | 0.3           | Controls the step size at each iteration. Smaller values make the model more robust but slower. |
| max_depth         | Positive integer     | 6             | Maximum depth of a tree. Higher values may lead to overfitting.                                 |
| subsample         | (0, 1]               | 1.0           | Fraction of samples used for each boosting round. Helps prevent overfitting.                    |
| colsample_bytree  | (0, 1]               | 1.0           | Fraction of features used for each boosting round. Helps prevent overfitting.                   |
| gamma             | Non-negative float   | 0.0           | Minimum loss reduction required to make a further partition on a leaf node.                  

We will try different values of these hyperparameters to see if we can improve model performance. This can be done systematically using a combination of [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) and [random search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search). First, we initialize an instance of the `XGBRegressor` class and define a distribution of parameter values to sample from.

In [ ]:
# Initialize an XGBoost model with default parameters
model = xgb.XGBRegressor()

# Parameter distributions
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.03, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

We could also use continuous distributions from [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html), but for simplicity here we'll stick to lists of discrete values.

Now we will search the hyperparameter space by taking random samples from `param_dist`. For each set of parameters sampled, we will benchmark the model performance using five-fold cross validation *within* the training set. Note we are not touching the testing set. We set `n_iter=50` so our final set of parameters will be the best set out of 50 that we try. Finally, note that we set `n_jobs=1`. This disables parallel processing for the cross validation. XGBoost will be using all of our cores to fit the model, so there's no point trying to parallelize the CV too. Depending on your machine, the next code cell could take a few minutes to run. If it's taking too long, consider reducing `n_iter`.

In [ ]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    n_jobs=1,
    random_state=42
)

# Fit the model to the training data
random_search.fit(X_train, y_train);

Now let's extract the set of parameters that produced the lowest cross-validated MSE on the training set. These parameters will be used to train a model on the full training set.

In [ ]:
# Get the best hyperparameters
best_params = random_search.best_params_

# Train the final model with the best hyperparameters
model_tuned = xgb.XGBRegressor(**best_params)
model_tuned.fit(X_train, y_train);

# Print final hyperparameter values
for key, value in best_params.items():
    print(key, value)

Note that cross validation selected the highest number of estimators in our distribution of candidate values. When the selected value lies at the boundary of the candidate region like this, it can be an indication that we should extend the range of candidate values. Consider trying some different values once you reach the end of the notebook.

## Model testing

We now have two candidate XGBoost models to predict sensible heat flux $H_s$: the first with default settings; the second with hyperparameters chosen through random search and cross validation on the training set. To evaluate their performance, we will compute RMSE and R<sup>2</sup> metrics on the testing set (which we haven't looked at thus far).

In [ ]:
# Make predictions on test set with each model
y_pred_default = model_default.predict(X_test)
y_pred_tuned = model_tuned.predict(X_test)

# Compute RMSE
rmse_default = np.sqrt(mean_squared_error(y_test, y_pred_default))
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

# Compute R^2
r2_default = r2_score(y_test, y_pred_default)
r2_tuned = r2_score(y_test, y_pred_tuned)

# Compare model performance
print(f'RMSE (default): {rmse_default:.2f} W/m2')
print(f'RMSE (tuned):   {rmse_tuned:.2f} W/m2')
print(f'R^2 (default):  {r2_default:.3f}')
print(f'R^2 (tuned):    {r2_tuned:.3f}')

In this example, tuning the hyperparameters appears to have given us a small boost in performance compared with the default algorithm. Note that these are probably not the best hyperparameters for this dataset. A more exhaustive search of the parameter space would increase our chances of finding better parameters, but at the cost of more computation time.

There is also no guarantee that a model with better cross-validated RMSE on the training set will perform better on the testing set. The gains in model performance that can be achieved through parameter tuning will vary from dataset to dataset. Other factors, such as the volume of data and the construction and selection of features (feature engineering) often play a bigger role in training a performant model.

Finally, note the difference in performance between the default model on the training set and the testing set.

In [ ]:
# Two panels side by side
fig, ax = plt.subplots(ncols=2)

# Range of values to display
lim = [-80, 80]
ax[0].set_xlim(lim)
ax[1].set_xlim(lim)
ax[0].set_ylim(lim)
ax[1].set_ylim(lim)

# Fix aspect ratio 1:1
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')

# Plot residuals on training set
ax[0].plot(y_train, y_pred_train, 'bo')
ax[0].axline((0, 0), slope=1, label='y=x')
ax[0].set_title('Training')
ax[0].set_xlabel('H_s true')
ax[0].set_ylabel('H_s predicted')
ax[0].legend()
ax[0].annotate(f'R^2: {r2:.3f}', (-70, 40))

# Plot residuals on testing set
ax[1].plot(y_test, y_pred_default, 'bo')
ax[1].axline((0, 0), slope=1, label='y=x')
ax[1].set_title('Testing')
ax[1].set_xlabel('H_s true')
ax[1].legend()
ax[1].annotate(f'R^2: {r2_default:.3f}', (-70, 40));

Note how much worse the model performs on the unseen data in the testing set. This example illustrates why we shouldn't rely on performance metrics computed on the training set when evaluating models.

## Feature importance

As with Random Forest, XGBoost models can tell us something about the relative importance of the features used to the target variable. XGBoost allows us to compute the feature importances using three different methods:

* `weight` (frequency): The weight of a feature represents the number of times that feature appears in the decision trees of the XGBoost model. Higher weight indicates greater influence.
* `gain` (average gain): Gain measures the average improvement in the model's loss function achieved by splits involving a specific feature. A higher gain value signifies a more crucial role in prediction accuracy.
* `cover` (average coverage): Cover quantifies the relative number of observations associated with leaf-node splits involving a feature. It's expressed as a percentage of the total observations.

We will use the `gain` method here. XGBoost includes functions for computing and plotting the feature importances that we will use below.

In [ ]:
# Plot feature importances for each model
fig, ax = plt.subplots(nrows=2)
xgb.plot_importance(
    booster=model_default,
    ax=ax[0],
    importance_type='gain',
    show_values=False,
    ylabel='Default',
    xlabel=None,
    xlim=(0,700)
)
xgb.plot_importance(
    booster=model_tuned,
    ax=ax[1],
    importance_type='gain',
    show_values=False,
    ylabel='Tuned',
    title=None,
    xlim=(0,700)
);

From these plots we can see that tuning the hyperparameters doesn't make much qualitative difference to the feature importances. Note that the `gain` metric (F score) is lower across the board for the model with tuned hyperparameters. Recall that `gain` measures the average improvement in loss **per split** in the model. Since we know that the tuned model has a lower RMSE than the default model, it must be using many more splits (deeper trees and more of them) to achieve its higher overall performance.

The results here are a little surprising from a physical point of view: we would expect `t2` and `t10` to be the most important features for explaining $H_s$, as the temperature difference between them is believed to be a driving force behind the turbulent fluxes. To investigate, we can check the correlation between 2m humidity `q2` and 2m temperature `t2`.

In [ ]:
# Compute Pearson correlation coefficient
df[['t2','q2']].corr()

Since these variables are very highly correlated, it is possible that the XGBoost models are using `q2` as a proxy for 2m temperature. This could imply that `q2` is measured more accurately than `t2`, in which case it might be a more reliable indicator of the true 2m temperature than the feature `t2`.